In [1]:
import csv
import cv2
import numpy as np
from sklearn.utils import shuffle

%matplotlib inline

base_dir = '/home/bibagimon/nanodegree/data'

def read_csv_data(path):
    lines = []
    
    with open(path) as csvfile:
        reader = csv.reader(csvfile)

        for line in reader:
            lines.append(line)
    return lines


def read_driving_log(path, images, measurements, augment=False):
    data = read_csv_data(path)

    for line in data:        
        measurement = float(line[3])

        for i, correction_factor in zip(range(3), [0, 0.2, -0.2]):
            source_path = line[i]
            
            image_bgr = cv2.imread(source_path)
            cropped = image_bgr[60:-10,20:-20]
            image_rgb = cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB)
            
            image = cv2.resize(image_rgb, (140, 50), interpolation=cv2.INTER_AREA)
            images.append(image)            
            measurements.append(measurement + correction_factor)
            
        if augment:
            images.append(np.fliplr(image))
            measurements.append(-measurement)           
    
tmp = []
for track, augment in zip(['track1_normal', 'track1_reverse', 'track2_normal'], [True, True, False]):
    path = '{}/{}/driving_log.csv'.format(base_dir, track)    
    tmp.append(read_csv_data(path))
    
driving_log = np.concatenate(tmp)
np.random.shuffle(driving_log)

In [2]:
print(len(driving_log))

8342


In [3]:
n_train = int(0.8 * len(driving_log))
train, valid = driving_log[:n_train], driving_log[n_train:]

print(train.shape)
print(valid.shape)

(6673, 7)
(1669, 7)


In [8]:
from keras.models import Sequential
from keras.layers import Flatten, Dense,Lambda, Cropping2D, Dropout, Conv2D

from keras.layers.pooling import MaxPooling2D

model = Sequential()
#model.add(Cropping2D(cropping=((60,20), (0,0)), input_shape=(160,320, 3)))
model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(50,140, 3)))
model.add(Conv2D(24, (5, 5), strides=(2, 2), activation='relu'))
model.add(Conv2D(36, (5, 5), strides=(2, 2), activation='relu'))
model.add(Conv2D(48, (5, 5), strides=(2, 2), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
#model.add(Convolution2D(64, 3, 3, activation='relu'))

model.add(Flatten())
model.add(Dense(1164))
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))


model.compile(loss='mse', optimizer='adam')

# Generators
from generator import DataGenerator

import random

def read_image_rgb(source_path):
    image_bgr = cv2.imread(source_path)    
    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
    
    return image_rgb

def generator(driving_log, batch_size, shape=(50, 140)):    
    correction_factor = [0, 0.2, -0.2]
    
    # Create empty arrays to contain batch of features and labels#
    batch_images = np.zeros((batch_size, shape[0], shape[1], 3))
    batch_measurements = np.zeros((batch_size, 1))
    
    while True:
        i = 0
        while i < batch_size:
            # choose random index in features
            log = random.choice(driving_log)            
            img_index = random.choice(range(0, 3))
            
            source_path = log[img_index]
            
            image_rgb = read_image_rgb(source_path)
            cropped = image_rgb[60:-10,20:-20]
            image = cv2.resize(cropped, (140, 50), interpolation=cv2.INTER_AREA)
            
            measurement = float(log[3]) + correction_factor[img_index]
            
            batch_images[i] = image
            batch_measurements[i] = measurement
            
            batch_images[i+1] = np.fliplr(image)
            batch_measurements[i+1] = -measurement
            
            i += 2
            
        yield batch_images, batch_measurements

batch_size=64
steps_per_epoch=int(train.shape[0]/batch_size)
validation_steps=int(valid.shape[0]/batch_size)
model.fit_generator(generator(train, batch_size),
                    validation_data=generator(valid, batch_size),
                    steps_per_epoch=steps_per_epoch, validation_steps=validation_steps, 
                    epochs=50
                   )
model.save('model.h5')


Epoch 1/50
104/104 [==============================] - 5s 52ms/step - loss: 0.1204 - val_loss: 0.0983
Epoch 2/50
104/104 [==============================] - 5s 46ms/step - loss: 0.0897 - val_loss: 0.0984
Epoch 3/50
104/104 [==============================] - 5s 45ms/step - loss: 0.1032 - val_loss: 0.0898
Epoch 4/50
104/104 [==============================] - 5s 46ms/step - loss: 0.1007 - val_loss: 0.0928
Epoch 5/50
104/104 [==============================] - 5s 46ms/step - loss: 0.0983 - val_loss: 0.0780
Epoch 6/50
104/104 [==============================] - 5s 46ms/step - loss: 0.0957 - val_loss: 0.0801
Epoch 7/50
104/104 [==============================] - 5s 47ms/step - loss: 0.0878 - val_loss: 0.0740
Epoch 8/50
104/104 [==============================] - 5s 46ms/step - loss: 0.0844 - val_loss: 0.0736
Epoch 9/50
104/104 [==============================] - 5s 46ms/step - loss: 0.0774 - val_loss: 0.0709
Epoch 10/50
104/104 [==============================] - 5s 46ms/step - loss: 0.0832 - val_lo

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(1, 3, figsize=(20, 5))
fig.subplots_adjust(hspace=0.1, wspace=0.1)
axs = axs.ravel()

for img, i in zip([("Left", 1), ("Center", 0), ("Right", 2)], range(0, 3)):
    source_path = train[0][0]
    
    axs[i].set_title(img[0])
    axs[i].imshow(read_image_rgb(source_path))